<a href="https://colab.research.google.com/github/enelene/Walmart-Recruiting---Store-Sales-Forecasting/blob/main/notebooks/NBEATSx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
# Add the directory containing your script to the Python path
sys.path.append('drive/MyDrive/Colab_Notebooks/Walmart-Recruiting---Store-Sales-Forecasting-main')

In [3]:
!pip install -r  drive/MyDrive/Colab_Notebooks/Walmart-Recruiting---Store-Sales-Forecasting-main/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.2/261.2 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import pandas as pd
import numpy as np
import torch
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATSx # 1. Import NBEATSx
from neuralforecast.losses.pytorch import MAE

In [6]:
import os
import warnings
import wandb
import gc
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import WandbLogger

In [2]:
def wmae(y_true, y_pred, is_holiday):
    """Computes the Weighted Mean Absolute Error."""
    weights = np.where(np.array(is_holiday, dtype=bool), 5, 1)
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)


In [3]:
try:
    train_df = pd.read_csv('drive/MyDrive/Colab_Notebooks/Walmart-Recruiting---Store-Sales-Forecasting-main/data/train_final.csv', parse_dates=['Date'])
    validation_df = pd.read_csv('drive/MyDrive/Colab_Notebooks/Walmart-Recruiting---Store-Sales-Forecasting-main/data/validation_final.csv', parse_dates=['Date'])
    test_df = pd.read_csv('drive/MyDrive/Colab_Notebooks/Walmart-Recruiting---Store-Sales-Forecasting-main/data/test_final.csv', parse_dates=['Date'])
    print("Successfully loaded all pre-split data files.")
except KeyError:
    print("ERROR: Make sure you have uploaded all three required CSV files.")
    raise

Successfully loaded all pre-split data files.


In [7]:
HORIZON = 39
INPUT_SIZE = 52
WANDB_PROJECT = "Walmart-Sales-Forecasting-Comparison"

In [8]:
def prepare_for_nf(df, is_train=True):
    df_copy = df.copy()
    df_copy['unique_id'] = df_copy['Store'].astype(str) + '_' + df_copy['Dept'].astype(str)
    df_copy.rename(columns={'Date': 'ds'}, inplace=True)
    if is_train and 'Weekly_Sales' in df_copy.columns:
        df_copy.rename(columns={'Weekly_Sales': 'y'}, inplace=True)
    return df_copy

In [9]:

# --- Data Preparation (no change) ---
nf_train_df = prepare_for_nf(train_df)
nf_validation_df = prepare_for_nf(validation_df)


In [10]:
# --- 2. Define Exogenous Variables ---
futr_exog_list = [
    'IsHoliday', 'Month', 'IsBlackFridayWeek', 'IsLaborDayWeek',
    'WeekOfYear', 'Day', 'IsChristmasWeek', 'IsSuperBowlWeek', 'Year'
]
futr_exog_list = [f for f in futr_exog_list if f in train_df.columns]
print(f"Using {len(futr_exog_list)} future exogenous features for NBEATSx.")


Using 9 future exogenous features for NBEATSx.


In [11]:
nf_test_df = prepare_for_nf(test_df, is_train=False)


In [ ]:
with wandb.init(project=WANDB_PROJECT, name="NBEATSx-Experiment", job_type="train-validate") as run:
    gc.collect()
    torch.cuda.empty_cache()

    # Create a logger to stream training metrics to our W&B run
    wandb_logger = WandbLogger(log_model=False, experiment=run)

    models = [
        NBEATSx(
            h=HORIZON,
            input_size=INPUT_SIZE,
            futr_exog_list=futr_exog_list, # <-- Pass the feature list here
            stack_types=['trend', 'seasonality', 'identity'],
            n_blocks=[3, 4, 3],
            mlp_units=[[256, 256]] * 3,
            max_steps=1000,
            early_stop_patience_steps=10,
            loss=MAE(),
            scaler_type='robust',
            random_seed=42,
            logger=wandb_logger # <-- Attach the logger to the model
        )
    ]

    # Train the model
    nf = NeuralForecast(models=models, freq='W-FRI')
    nf.fit(df=nf_train_df, val_size=HORIZON)
    print("Initial model training complete.")

    future_validation_df = nf.make_future_dataframe()
    future_validation_df = pd.merge(
        future_validation_df,
        nf_validation_df[['unique_id', 'ds'] + futr_exog_list],
        on=['unique_id', 'ds'],
        how='left' # Use left merge to keep all future dates
    )

    # Fill null values in exogenous features using forward fill within each unique_id
    for col in futr_exog_list:
        # Convert to numeric before ffill to avoid type issues with NaNs
        future_validation_df[col] = pd.to_numeric(future_validation_df[col], errors='coerce')
        future_validation_df[col] = future_validation_df.groupby('unique_id')[col].ffill()


    # Fill any remaining null values (those at the start of a series) with 0
    for col in futr_exog_list:
        future_validation_df[col] = future_validation_df[col].fillna(0)

    # Provide the future dataframe with the exogenous features for prediction
    predictions_df = nf.predict(futr_df=future_validation_df)
    eval_df = pd.merge(nf_validation_df, predictions_df, on=['unique_id', 'ds'])

    # Calculate and log metrics
    wmae_score = wmae(eval_df['y'], eval_df['NBEATSx'], eval_df['IsHoliday'])
    mae_score = np.mean(np.abs(eval_df['y'] - eval_df['NBEATSx']))
    rmse_score = np.sqrt(np.mean((eval_df['y'] - eval_df['NBEATSx'])**2))

    print(f"\nValidation WMAE: {wmae_score:.4f}")
    print(f"Validation MAE:  {mae_score:.4f}")
    print(f"Validation RMSE: {rmse_score:.4f}")

    run.log({
        "validation_wmae": wmae_score,
        "validation_mae": mae_score,
        "validation_rmse": rmse_score
    })

    # --- Generate and Log a Sample Forecast Plot ---
    try:
        sample_id = '4_1' # A sample store-dept to visualize
        sample_hist_df = nf_train_df[nf_train_df['unique_id'] == sample_id]
        sample_eval_df = eval_df[eval_df['unique_id'] == sample_id]

        plt.figure(figsize=(14, 7))
        plt.plot(sample_hist_df['ds'], sample_hist_df['y'], label='History', color='black')
        plt.plot(sample_eval_df['ds'], sample_eval_df['y'], label='Actual (Validation)', color='blue')
        plt.plot(sample_eval_df['ds'], sample_eval_df['NBEATSx'], label='Forecast (NBEATSx)', color='red', linestyle='--')
        plt.title(f'NBEATSx Forecast vs. Actual for {sample_id}')
        plt.legend()
        plt.grid(True)
        run.log({"Sample Validation Plot": wandb.Image(plt)})
        plt.close() # Close plot to free memory
        print(f"\nLogged a sample forecast plot for ID {sample_id} to W&B.")
    except Exception as e:
        print(f"Could not generate plot: {e}")

    # --- 4. Final Model Retraining & Submission ---
    print("\n--- Retraining final model on all available data ---")
    full_train_df = pd.concat([nf_train_df, nf_validation_df])

    final_model = [
        NBEATSx(
            h=HORIZON,
            input_size=INPUT_SIZE,
            futr_exog_list=futr_exog_list,
            stack_types=['trend', 'seasonality', 'identity'],
            n_blocks=[3, 4, 3],
            mlp_units=[[256, 256]] * 3,
            max_steps=1500, # Train a bit longer on the full data
            loss=MAE(),
            scaler_type='robust',
            random_seed=42
        )
    ]

    nf_final = NeuralForecast(models=final_model, freq='W-FRI')
    nf_final.fit(df=full_train_df) # No val_size needed
    print("Final model retraining complete.")

    # --- Generate Final Predictions ---
    print("\nGenerating predictions on the test set...")
    # Provide the test dataframe with its future exogenous features
    # Use make_future_dataframe for the final prediction as well
    future_test_df = nf_final.make_future_dataframe()
    future_test_df = pd.merge(
        future_test_df,
        nf_test_df[['unique_id', 'ds'] + futr_exog_list],
        on=['unique_id', 'ds'],
        how='left'
    )

    # Fill null values in exogenous features using forward fill within each unique_id for test data
    for col in futr_exog_list:
        future_test_df[col] = pd.to_numeric(future_test_df[col], errors='coerce')
        future_test_df[col] = future_test_df.groupby('unique_id')[col].ffill()


    final_predictions_df = nf_final.predict(futr_df=future_test_df)

    # --- Format and Save Submission File ---
    test_df['Id'] = test_df['Store'].astype(str) + '_' + test_df['Dept'].astype(str) + '_' + test_df['Date'].dt.strftime('%Y-%m-%d')

    # Merge predictions back into the original test_df structure
    # Need to ensure the merge keys match the prediction dataframe
    submission_df = pd.merge(
        test_df[['Id', 'Store', 'Dept', 'Date']], # Select relevant columns from original test_df
        final_predictions_df.rename(columns={'ds': 'Date', 'NBEATSx': 'Weekly_Sales'}),
        left_on=['Date', 'Store', 'Dept'], # Merge on original test_df columns
        right_on=['Date', 'Store', 'Dept'], # Merge on prediction df columns
        how='left'
    )


    final_submission = submission_df[['Id', 'Weekly_Sales']].copy()
    final_submission['Weekly_Sales'] = final_submission['Weekly_Sales'].clip(lower=0).fillna(0)
    submission_filename = 'nf_nbeatsx_submission.csv'
    final_submission.to_csv(submission_filename, index=False)
    print(f"\nSubmission file '{submission_filename}' created successfully.")

    # --- 5. Finalize Logging ---
    print("Logging submission file as a W&B Artifact...")
    artifact = wandb.Artifact('submission-file', type='submission')
    artifact.add_file(submission_filename)
    run.log_artifact(artifact)

    print("\n Experiment complete.")


wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
INFO:lightning_fabric.utilities.seed:Seed set to 42
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/neuralforecast/core.py:553: UserWarning: Validation set size is larger than the shorter time-series.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: WARNING `w

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]